In [1]:
import pandas as pd
import os
import math

In [2]:
"""Report Lab Graph Generations"""
from reportlab.lib import colors
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4,landscape
from reportlab.lib.units import inch
from reportlab.graphics.shapes import Drawing
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer,PageBreak
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.graphics.charts.piecharts import Pie
from reportlab.graphics.charts.linecharts import HorizontalLineChart
"""Import for Table"""
from reportlab.platypus import Table,TableStyle
def bulidTableFromDataFrame(myframe):
    width, height = landscape(A4)
    cols=list(myframe.columns)
    col_widths=[50 for i in range(len(cols))]
    data=[]
    
    data.append(cols)    
    for index,row in myframe.iterrows():
        myrow=list(myframe.iloc[index,:])
        data.append(myrow)
        #print(myrow)
    t=Table(data,colWidths=col_widths,hAlign='LEFT',vAlign="CENTER")
    t.setStyle(TableStyle([('GRID', (0,0), (-1,-1), 0.25, colors.black),
                           ('BOX', (0, 0), (-1, -1), 0.25, colors.white)
                          ]))
    
    return t


def myFirstPage(canvas, doc):
    canvas.saveState()
    canvas.setPageSize((11.75*inch, 8.25*inch))  
    canvas.restoreState()
def myLaterPages(canvas, doc):
    canvas.saveState()    
    canvas.setPageSize((11.75*inch, 8.25*inch))    
    canvas.restoreState()
def docBuild(filename,story):
    doc = SimpleDocTemplate(filename)
    Story = [Spacer(1,2*inch)]
    styles=getSampleStyleSheet()
    style = styles["Normal"]
    """  
    for i in range(100):
        bogustext = ("This is Paragraph number %s. " % i) *20
        p = Paragraph(bogustext, style)
        Story.append(p)
        Story.append(Spacer(1,0.2*inch))
    """
    doc.build(story, onFirstPage=myFirstPage, onLaterPages=myLaterPages)
    
def createPdf(filename,pagesize):
    pdf=SimpleDocTemplate(filename,pagesize=pagesize)
    return pdf
def getHorizontalLineChart(data):
    chart=HorizontalLineChart()
    chart.data=data
    drawing=Drawing(400,400)
    drawing.add(chart)
    return drawing
def createA4Canvas(filename):    
    mycanvas=canvas.Canvas(filename,pagesize=landscape(A4),bottomup =0)
    width, height = landscape(A4)
    width=width-1*inch
    height=height-1*inch
    c_x=1*inch
    c_y=1*inch
    myvalue={"width":width,"height":height,"canvas":mycanvas,'c_x':c_x,'c_y':c_y}
    return myvalue
def createPage(out_file_name,page_size=A4):    
    mycanvas=canvas.Canvas(out_file_name,pagesize=landscape(page_size),bottomup =0)
    width, height = landscape(page_size)
    width=width-1*inch
    height=height-1*inch
    c_x=1*inch
    c_y=1*inch
    myvalue={"width":width,"height":height,"canvas":mycanvas,'c_x':c_x,'c_y':c_y}
    return myvalue

In [3]:
"""Functions Related to Tranfering Data to Excel Sheet"""
def writeDataframe6(wbpath,df,sheetname,dfcols,xlcols,indexcol):    
    wb=opxl.load_workbook(wbpath)
    wsheet=wb[sheetname]    
    for index,row in df.iterrows():        
        myrow=int(row[indexcol])
        for idf,ixl in zip(dfcols,xlcols):
            wsheet.cell(row=myrow,column=ixl).value=row[idf]               
    wb.save(wbpath)
def saveDataFrame(dataframes,filepath,names):
    writer = pd.ExcelWriter(filepath, engine='xlsxwriter')
    for df,sname in zip(dataframes,names):
        df.to_excel(writer,sheet_name=sname)   
    writer.save()  

In [4]:
class typeAreach:
    def __init__(self,reach_name,fillHeight,Length):
        self.Name=reach_name
        self.h=fillHeight
        self.length=Length
        self.n=3
        self.b=4.3        
        a=(self.b+self.n*self.h)*self.h
        self.area=round(a,3)
        p=(self.b+2*self.h*math.sqrt(math.pow(self.n,2)+1))
        self.perimeter=round(p,3)
    def calcualteUnitBoq(self):
        pass        
    def display(self):
        print("Name={} h={} L={} a={} p={}".format(self.Name,self.h,self.length,self.area,self.perimeter))
        
        
    
        

In [5]:
def calculateExtensionLength(n,t):
    ext=2*n*t+4*t*(math.sqrt(math.pow(n,2)+1)-n)
    return ext
def calculateBoQ(h,b,n,L,wtype):
    myboq=[0 for x in range(0,15)]
    h1=round(h-0.2-0.1-0.003-0.1,3)
    a=(b+n*h1)*h1
    a=round(a,3)
    p=(b+2*h1*math.sqrt(math.pow(n,2)+1))
    p=round(p,3)
    """adding earth work"""
    bprofile=math.ceil(L/30.0)
    total_ew=a*L
    ew_carried_earth=total_ew*0.5
    ew_mechanical_excavator=total_ew*0.5
    ew_royalty=total_ew*0.5
    myboq[0]=bprofile
    myboq[1]=ew_carried_earth
    myboq[2]=ew_mechanical_excavator
    myboq[3]=ew_royalty    
    if wtype=='A':
        """adding filters"""
        t=0.1
        t_half=0.1/2.0
        extension1=2*n*t_half+4*(t_half)*(math.sqrt(math.pow(n,2)+1)-n)
        extension2=2*n*t+4*t*(math.sqrt(math.pow(n,2)+1)-n)        
        print("extension={} extension2={}".format(extension1,extension2))
        sf_length=p+extension1
        sandfilter_volume=sf_length*0.1*L
        anchor_length=2.0        
        geotex_width=p+extension2+anchor_length
        geotex_area=geotex_width*L
        extension3=calculateExtensionLength(n,0.153)
        gf_length=p+extension3
        gf_filter_vol=gf_length*0.1*L
        gf1=gf_filter_vol*0.5
        gf2=gf_filter_vol*0.5
        myboq[4]=sandfilter_volume
        myboq[5]=geotex_area
        myboq[6]=gf1
        myboq[7]=gf2
        """adding cc block"""
        extension4=calculateExtensionLength(n,0.203)
        block_piticing_length=p+extension4
        block_piticing_length=round(block_piticing_length,2)
        bno=math.ceil(block_piticing_length/0.4)
        block_per_meter=math.ceil(bno*2.5)
        block1=block_per_meter*L
        print("pitching_length={} nos={} bloc/m={} total={}".format(block_piticing_length,
                                                           bno,block_per_meter,
                                                                   block1))
        block2=math.ceil(10*(L/0.3))
        block_volume=block1*(0.4*0.4*0.2)+block2*(0.3*0.3*0.3)
        vo11=round(block_volume*0.5,3)
        cc_in_joint=(2*0.3*0.2*0.5)*L
        myboq[8]=block1
        myboq[9]=block2
        myboq[10]=vo11
        myboq[11]=vo11
        myboq[13]=cc_in_joint
    if wtype=='B5':
        h2=round(h-0.3,3)
        b1=b+2*n*0.3
        p1=(b1+2*h2*math.sqrt(math.pow(n,2)+1))
        p2=(b+2*h*math.sqrt(math.pow(n,2)+1))
        
        bag_at_top=math.ceil(1*L+1)
        bag_toe=math.ceil(1.5*(L/0.675)*2)
        total_bag=bag_at_top+bag_toe
        myboq[5]=round(p1*L,3)
        myboq[12]=total_bag
        myboq[14]=round(p2*L,3)
        print("h1={} area={} perimeter={} protection_type={} ".format(h1,a,p,wtype))
    return myboq
        
def calCulateReachAveragedFillHeight(input_df):
    area=input_df.iloc[-1,6]
    width=(input_df.iloc[-1,0]-input_df.iloc[0,0])*1000
    h=round((area/width),2)
    return h
    
    
def addReachAveragedFillHeightData(input_df):
    for index,row in input_df.iterrows():
        if index>0:
            h_avg=(input_df.iloc[index,2]+input_df.iloc[index-1,2])/2.0
            delx=0
            delA=0
            delx=(input_df.iloc[index,0]-input_df.iloc[index-1,0])*1000
            delA=delx*h_avg
            input_df.iloc[index,3]=h_avg
            input_df.iloc[index,4]=delx
            input_df.iloc[index,5]=delA
        if index==1:
            input_df.iloc[index,6]=delA
        if index>1:
            input_df.iloc[index,6]=delA+input_df.iloc[index-1,6]
            
            
    return input_df
    
def calcualteFillHeight(Area,sideSlope,topWidth):
    A=Area
    n=sideSlope
    b=topWidth
    t1=math.pow(b,2)
    t2=4*A*n
    t3=t1+t2
    t4=math.sqrt(t3)
    t5=t4-b
    t6=2*n
    h3=t5/t6
    #print("t1={} t2={} t3={} t4={} t5={} t6={} h={}".format(t1,t2,t3,t4,t5,t6,h3))
    #h1=(-b+math.sqrt(math.pow(b,2)+4*A**n))/(2.0*n)
    h=round(h3,2)
    return h
def addFillHeighttoDf(input_df):
    for index,row in input_df.iterrows():
        area=row['Area']
        h=calcualteFillHeight(area,3,4.3)
        print(h)
        input_df.iloc[index,2]=h
    return input_df
def testDf(input_df):
    mydfs=[]
    for index,row in input_df.iterrows():
        sheetName=row['SheetName']
        mydf=pd.read_excel(input_path,sheet_name=sheetName)       
        mydf.fillna(0,inplace=True)
        mydf1=addFillHeighttoDf(mydf)
        mydf2=addReachAveragedFillHeightData(mydf1)
        print(mydf2)
        mydfs.append(mydf2)
    return mydfs
def addFillHeight_to_Reach_Df(reach_df):
    mydfs=[]
    for index,row in reach_df.iterrows():
        sheetName=row['SheetName']        
        mydf=pd.read_excel(input_path,sheet_name=sheetName)       
        mydf.fillna(0,inplace=True)
        mydf1=addFillHeighttoDf(mydf)
        mydf2=addReachAveragedFillHeightData(mydf1)
        h=calCulateReachAveragedFillHeight(mydf2)
        reach_df.iloc[index,4]=h
        #print(mydf2)
        mydfs.append(mydf2)
    myvalues={'reach_data':reach_df,'fill_height_cal':mydfs}
    return myvalues
def createBoQDf(boq_tempalte,reach_df):
    for index,row in reach_df.iterrows():
        name=row['Reach']
        h=row['fill_height']
        length=row['Length']
        work_type=row['type']
        myboq=calculateBoQ(h,4.3,3,length,work_type)
        boq_tempalte[name]=myboq
    print(boq_tempalte)
    return boq_tempalte
def createPackageBoq(boq_df_corrected):
    myshape=boq_df_corrected.shape
    
    "ItemCode 	Description 	Unit 	Rate"
    ItemCode=list(boq_df_corrected['ItemCode'])
    Description=list(boq_df_corrected['Description'])
    Unit=list(boq_df_corrected['Unit'])
    Rate=list(boq_df_corrected['Rate'])
    zerolist=[0 for x in ItemCode]
    data={"ItemCode":ItemCode,"Description":Description,"Unit":Unit,"Rate":Rate,"Quantity":zerolist}
    #data={"ItemCode":ItemCode,"Description":Description,"Unit":Unit,"Rate":Rate}
    myboq=pd.DataFrame(data)
    #myboq=boq_df_corrected.iloc[:,0:4]
    return myboq
def findRechesForPacakge(reach_df,pack_name):
    myreaches=[]
    for index,row in reach_df.iterrows():
        reach=row['Reach']
        package=row['package']
        if package==pack_name:
            myreaches.append(reach)
    return myreaches  
def populatePacakgeBoQ(boq_df,reach_names,package_boq_df):
    n=4+len(reach_names)
    for reach in reach_names:
        mylist=list(boq_df_corrected[reach])
        package_boq_df[reach]=boq_df[reach]
    package_boq_df['Total_quantity']=package_boq_df.iloc[:,4:n].sum(axis=1)
    #package_boq_df['Amount']=package_boq_df['Rate']*package_boq_df['Total_quantity']
    
    return package_boq_df
def populatePacakgeBoQ2(boq_df,reach_names,package_boq_df):
    
    for reach in reach_names:
        total_q=list(package_boq_df['Quantity'])
        mylist=list(boq_df[reach])
        added_list=[total_q[i]+ mylist[i] for i in range(len( mylist))]
        package_boq_df['Quantity']=added_list
        
        #package_boq_df[reach]=boq_df[reach]
    #package_boq_df['Total_quantity']=package_boq_df.iloc[:,4:n].sum(axis=1)
    #package_boq_df['Amount']=package_boq_df['Rate']*package_boq_df['Total_quantity']
    
    return package_boq_df

def assemblePacakgeDf(boq_df_corrected,package_name_df,reach_df):
    packges=[]
    for index,row in package_name_df.iterrows():
        pname=row['name']
        print("package={}".format(pname))
        myboq=createPackageBoq(boq_df_corrected)
        print(myboq)
        reaches=findRechesForPacakge(reach_df, pname)       
        print("reaches={}".format(reaches))
        if reaches:
            myboq=populatePacakgeBoQ2(boq_df_corrected,reaches,myboq)
            print(myboq)
        packges.append(myboq)
    return packges
        
    
        
        
    

In [6]:
#myfolder=r'E:\Website_26_07_2020\cmis6\Civilworks cost\Slope Protection'
myfolder=r'E:\Website_24_11_2020\cmis6\cmis6\Civilworks cost\Protective Work Latest\Flood Fuse -14 (For Working)-2'
input_path=os.path.join(myfolder,'Input.xlsx')
output_path=os.path.join(myfolder,'Output.xlsx')
output_pdf=os.path.join(myfolder,'output.pdf')
"""creating pdf"""
"""   
width,height=landscape(A4)
psize=(width,height)
createPdf
mypdf=createPdf(output_pdf,psize)
mypdf.pagesize=landscape(A4)
pdf_elems=[]
data=[(3,18,20),(14,12,21)]
drawing=getHorizontalLineChart(data)
pdf_elems.append(drawing)
"""



sheetName='Main'
reach_df=pd.read_excel(input_path,sheet_name=sheetName)
reach_df.fillna(0,inplace=True)
reach_df
myvalues=addFillHeight_to_Reach_Df(reach_df)
reach_df_corrected=myvalues['reach_data']
mydfs=myvalues['fill_height_cal']
    #mydfs=testDf(reach_df)
sheetName="BoQ"
boq_df=pd.read_excel(input_path,sheet_name=sheetName)
boq_df.fillna(0,inplace=True)
"""calcualting Boq"""
boq_df_corrected=createBoQDf(boq_df,reach_df_corrected)
"""Package_df"""
sheetName='Packages'
package_name_df=pd.read_excel(input_path,sheet_name=sheetName)
package_name_df.fillna(0,inplace=True)
package_name_df
packages=assemblePacakgeDf(boq_df_corrected,package_name_df,reach_df_corrected)

myframes=[]
mynames=[]

1.41
1.41
1.41
1.41
1.41
3.31
3.31
3.31
3.31
3.31
3.31
3.31
3.31
3.31
3.31
1.41
1.41
1.41
1.41
1.41
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
1.5
1.5
1.5
1.5
1.5
1.41
1.41
1.41
1.41
1.41
1.5
1.5
1.5
1.5
1.5
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
0.9
0.9
1.16
1.14
1.6
1.62
1.62
1.17
1.3
1.32
1.31
1.25
1.09
1.09
1.09
1.16
1.52
1.41
1.41
h1=1.007 area=7.372 perimeter=10.669 protection_type=B5 
extension=0.33245553203367595 extension2=0.6649110640673519
pitching_length=24.03 nos=61 bloc/m=153 total=32283
extension=0.33245553203367595 extension2=0.6649110640673519
pitching_length=24.03 nos=61 bloc/m=153 total=7650
h1=1.007 area=7.372 perimeter=10.669 protection_type=B5 
extension=0.33245553203367595 extension2=0.6649110640673519
pitching_length=15.75 nos=40 bloc/m=100 total=2000
extension=0.33245553203367595 extension2=0.6649110640673519
pitching_length=15.75 nos=40 bloc/m=100 total=2000
h1=1.097 area=8.327 perimeter=11.238 protection_type=B5 
h1=1.007 area=7.372 perimeter=10.669 protection_

In [7]:
boq_df_corrected

,ItemCode,Description,Unit,Rate,S1,S2U,S2D,S3,S4U,S4D,S5,S6,S7,S8U,S8D,KP30R1A,KP30R2A,KP30R3A,KP30R4B5
0,16-100,Erection of bamboo profile,No,367.41,25.000,8.000000,2.000000,39.000,1.000000,1.000000,29.000,39.000,129.000,1.000000,1.000000,2.000000,17.000000,38.000000,24.000
1,16-220-10,Earth work by carried earth (by truck/boat or ...,Cum,429.88,2668.664,3993.386000,946.300000,4294.190,145.180000,145.180000,3538.975,4238.900,16029.475,145.180000,145.180000,73.389000,1406.580000,3457.440000,2194.850
2,16-240-10,Earth work by Mechanical Excavator (Long Boon)...,Cum,157.70,2668.664,3993.386000,946.300000,4294.190,145.180000,145.180000,3538.975,4238.900,16029.475,145.180000,145.180000,73.389000,1406.580000,3457.440000,2194.850
3,16-180,Royalty of specified earth taken from private ...,Cum,16.97,2668.664,3993.386000,946.300000,4294.190,145.180000,145.180000,3538.975,4238.900,16029.475,145.180000,145.180000,73.389000,1406.580000,3457.440000,2194.850
4,40-550-30,Supplying and laying sand as filter layers FM...,Cum,1267.96,0.000,485.668312,115.087278,0.000,29.464911,29.464911,0.000,0.000,0.000,29.464911,29.464911,39.654823,499.772232,1147.155020,0.000
5,40-500-40,Supplying and placing non-woven needle punched...,Sqm,250.13,9499.066,5348.831235,1267.495553,15285.099,341.298221,341.298221,11636.046,15088.295,52704.446,341.298221,341.298221,515.503464,6187.274643,14083.900392,8697.189
6,40-530-20,Supplying and laying shingles and peagravels a...,Cum,5771.61,0.000,250.059412,59.255785,0.000,15.417314,15.417314,0.000,0.000,0.000,15.417314,15.417314,21.573801,267.350005,611.929580,0.000
7,40-530-30,Supplying and laying shingles and peagravels a...,Cum,6135.23,0.000,250.059412,59.255785,0.000,15.417314,15.417314,0.000,0.000,0.000,15.417314,15.417314,21.573801,267.350005,611.929580,0.000
8,40-190-40,Manufacturing and supplying C.C. blocks in lea...,Nos,457.33,0.000,32283.000000,7650.000000,0.000,2000.000000,2000.000000,0.000,0.000,0.000,2000.000000,2000.000000,2805.000000,35700.000000,81760.000000,0.000
9,40-190-50,Manufacturing and supplying C.C. blocks in lea...,Nos,380.95,0.000,7034.000000,1667.000000,0.000,667.000000,667.000000,0.000,0.000,0.000,667.000000,667.000000,1700.000000,17000.000000,37334.000000,0.000


In [8]:
myframes.append(reach_df_corrected)
mynames.append('Reach_input')
k=1
for df in mydfs:    
    print(df)
    h=calCulateReachAveragedFillHeight(df)    
    print("h={}".format(h))
    name="R"+str(k)
    length=(df.iloc[-1,0]-df.iloc[0,0])*1000
    length=round(length,2)
    myreach=typeAreach(name,h,length)
    myreach.display()
    myframes.append(df)
    mynames.append(name)    
    k +=1
myframes.append(boq_df_corrected)
mynames.append("BoQ")

 
for index,row in package_name_df.iterrows():
    p=row['name']
    p1=p.replace("/","_")
    p2=p1.replace("-","_")
    myframes.append(packages[index])
    mynames.append(p2)
    print(p2)   
saveDataFrame(myframes,output_path,mynames)

   chainage   Area     h  h_avg  delx   delA  cumDelA
0    12.430  12.06  1.41   0.00   0.0   0.00     0.00
1    12.440  12.06  1.41   1.41  10.0  14.10    14.10
2    12.450  12.06  1.41   1.41  10.0  14.10    28.20
3    12.470  12.06  1.41   1.41  20.0  28.20    56.40
4    12.481  12.06  1.41   1.41  11.0  15.51    71.91
h=1.41
Name=R1 h=1.41 L=51.0 a=12.027 p=13.218
   chainage   Area     h  h_avg  delx   delA  cumDelA
0    12.430  47.21  3.31   0.00   0.0   0.00     0.00
1    12.440  47.21  3.31   3.31  10.0  33.10    33.10
2    12.450  47.21  3.31   3.31  10.0  33.10    66.20
3    12.470  47.21  3.31   3.31  20.0  66.20   132.40
4    12.481  47.21  3.31   3.31  11.0  36.41   168.81
h=3.31
Name=R2 h=3.31 L=51.0 a=47.101 p=25.234
   chainage   Area     h  h_avg  delx   delA  cumDelA
0    12.430  47.21  3.31   0.00   0.0   0.00     0.00
1    12.440  47.21  3.31   3.31  10.0  33.10    33.10
2    12.450  47.21  3.31   3.31  10.0  33.10    66.20
3    12.470  47.21  3.31   3.31  20.0  66.

In [9]:
story=[]
for myframe in myframes:
    tbl=bulidTableFromDataFrame(myframe)
    story.append(tbl)
    story.append(PageBreak())
    #story.append(Spacer(1,0.2*inch))
    

docBuild(output_pdf,story)

In [10]:
boq_df_corrected

,ItemCode,Description,Unit,Rate,S1,S2U,S2D,S3,S4U,S4D,S5,S6,S7,S8U,S8D,KP30R1A,KP30R2A,KP30R3A,KP30R4B5
0,16-100,Erection of bamboo profile,No,367.41,25.000,8.000000,2.000000,39.000,1.000000,1.000000,29.000,39.000,129.000,1.000000,1.000000,2.000000,17.000000,38.000000,24.000
1,16-220-10,Earth work by carried earth (by truck/boat or ...,Cum,429.88,2668.664,3993.386000,946.300000,4294.190,145.180000,145.180000,3538.975,4238.900,16029.475,145.180000,145.180000,73.389000,1406.580000,3457.440000,2194.850
2,16-240-10,Earth work by Mechanical Excavator (Long Boon)...,Cum,157.70,2668.664,3993.386000,946.300000,4294.190,145.180000,145.180000,3538.975,4238.900,16029.475,145.180000,145.180000,73.389000,1406.580000,3457.440000,2194.850
3,16-180,Royalty of specified earth taken from private ...,Cum,16.97,2668.664,3993.386000,946.300000,4294.190,145.180000,145.180000,3538.975,4238.900,16029.475,145.180000,145.180000,73.389000,1406.580000,3457.440000,2194.850
4,40-550-30,Supplying and laying sand as filter layers FM...,Cum,1267.96,0.000,485.668312,115.087278,0.000,29.464911,29.464911,0.000,0.000,0.000,29.464911,29.464911,39.654823,499.772232,1147.155020,0.000
5,40-500-40,Supplying and placing non-woven needle punched...,Sqm,250.13,9499.066,5348.831235,1267.495553,15285.099,341.298221,341.298221,11636.046,15088.295,52704.446,341.298221,341.298221,515.503464,6187.274643,14083.900392,8697.189
6,40-530-20,Supplying and laying shingles and peagravels a...,Cum,5771.61,0.000,250.059412,59.255785,0.000,15.417314,15.417314,0.000,0.000,0.000,15.417314,15.417314,21.573801,267.350005,611.929580,0.000
7,40-530-30,Supplying and laying shingles and peagravels a...,Cum,6135.23,0.000,250.059412,59.255785,0.000,15.417314,15.417314,0.000,0.000,0.000,15.417314,15.417314,21.573801,267.350005,611.929580,0.000
8,40-190-40,Manufacturing and supplying C.C. blocks in lea...,Nos,457.33,0.000,32283.000000,7650.000000,0.000,2000.000000,2000.000000,0.000,0.000,0.000,2000.000000,2000.000000,2805.000000,35700.000000,81760.000000,0.000
9,40-190-50,Manufacturing and supplying C.C. blocks in lea...,Nos,380.95,0.000,7034.000000,1667.000000,0.000,667.000000,667.000000,0.000,0.000,0.000,667.000000,667.000000,1700.000000,17000.000000,37334.000000,0.000


In [11]:
reach_df_corrected

,Reach,Start,Finish,Length,fill_height,type,package,SheetName,rowindex
0,S1,0.000,0.724,724,1.41,B5,SUNM/PW-07,S1,2
1,S2U,0.724,0.935,211,3.31,A,SUNM/PW-07,S2U,3
2,S2D,0.955,1.105,50,3.31,A,SUNM/PW-07,S2D,4
3,S3,1.105,2.180,1165,1.41,B5,SUNM/PW-07,S3,5
4,S4U,4.500,4.520,20,2.00,A,SUNM/PW-07,S4U,6
5,S4D,4.535,4.555,20,2.00,A,SUNM/PW-07,S4D,7
6,S5,4.600,7.800,850,1.50,B5,SUNM/PW-07,S5,8
7,S6,9.200,10.350,1150,1.41,B5,SUNM/PW-07,S6,9
8,S7,25.200,36.300,3850,1.50,B5,SUNM/PW-07,S7,10
9,S8U,35.030,35.050,20,2.00,A,SUNM/PW-07,S8U,11


In [12]:
boq_df

,ItemCode,Description,Unit,Rate,S1,S2U,S2D,S3,S4U,S4D,S5,S6,S7,S8U,S8D,KP30R1A,KP30R2A,KP30R3A,KP30R4B5
0,16-100,Erection of bamboo profile,No,367.41,25.000,8.000000,2.000000,39.000,1.000000,1.000000,29.000,39.000,129.000,1.000000,1.000000,2.000000,17.000000,38.000000,24.000
1,16-220-10,Earth work by carried earth (by truck/boat or ...,Cum,429.88,2668.664,3993.386000,946.300000,4294.190,145.180000,145.180000,3538.975,4238.900,16029.475,145.180000,145.180000,73.389000,1406.580000,3457.440000,2194.850
2,16-240-10,Earth work by Mechanical Excavator (Long Boon)...,Cum,157.70,2668.664,3993.386000,946.300000,4294.190,145.180000,145.180000,3538.975,4238.900,16029.475,145.180000,145.180000,73.389000,1406.580000,3457.440000,2194.850
3,16-180,Royalty of specified earth taken from private ...,Cum,16.97,2668.664,3993.386000,946.300000,4294.190,145.180000,145.180000,3538.975,4238.900,16029.475,145.180000,145.180000,73.389000,1406.580000,3457.440000,2194.850
4,40-550-30,Supplying and laying sand as filter layers FM...,Cum,1267.96,0.000,485.668312,115.087278,0.000,29.464911,29.464911,0.000,0.000,0.000,29.464911,29.464911,39.654823,499.772232,1147.155020,0.000
5,40-500-40,Supplying and placing non-woven needle punched...,Sqm,250.13,9499.066,5348.831235,1267.495553,15285.099,341.298221,341.298221,11636.046,15088.295,52704.446,341.298221,341.298221,515.503464,6187.274643,14083.900392,8697.189
6,40-530-20,Supplying and laying shingles and peagravels a...,Cum,5771.61,0.000,250.059412,59.255785,0.000,15.417314,15.417314,0.000,0.000,0.000,15.417314,15.417314,21.573801,267.350005,611.929580,0.000
7,40-530-30,Supplying and laying shingles and peagravels a...,Cum,6135.23,0.000,250.059412,59.255785,0.000,15.417314,15.417314,0.000,0.000,0.000,15.417314,15.417314,21.573801,267.350005,611.929580,0.000
8,40-190-40,Manufacturing and supplying C.C. blocks in lea...,Nos,457.33,0.000,32283.000000,7650.000000,0.000,2000.000000,2000.000000,0.000,0.000,0.000,2000.000000,2000.000000,2805.000000,35700.000000,81760.000000,0.000
9,40-190-50,Manufacturing and supplying C.C. blocks in lea...,Nos,380.95,0.000,7034.000000,1667.000000,0.000,667.000000,667.000000,0.000,0.000,0.000,667.000000,667.000000,1700.000000,17000.000000,37334.000000,0.000


In [13]:
reach_df

,Reach,Start,Finish,Length,fill_height,type,package,SheetName,rowindex
0,S1,0.000,0.724,724,1.41,B5,SUNM/PW-07,S1,2
1,S2U,0.724,0.935,211,3.31,A,SUNM/PW-07,S2U,3
2,S2D,0.955,1.105,50,3.31,A,SUNM/PW-07,S2D,4
3,S3,1.105,2.180,1165,1.41,B5,SUNM/PW-07,S3,5
4,S4U,4.500,4.520,20,2.00,A,SUNM/PW-07,S4U,6
5,S4D,4.535,4.555,20,2.00,A,SUNM/PW-07,S4D,7
6,S5,4.600,7.800,850,1.50,B5,SUNM/PW-07,S5,8
7,S6,9.200,10.350,1150,1.41,B5,SUNM/PW-07,S6,9
8,S7,25.200,36.300,3850,1.50,B5,SUNM/PW-07,S7,10
9,S8U,35.030,35.050,20,2.00,A,SUNM/PW-07,S8U,11


In [14]:
myboq=[0 for x in range(0,15)]
myboq

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [15]:
ext=calculateExtensionLength(3,0.153)
extension4=calculateExtensionLength(3,0.203)
print(extension4)

1.349769460056724


In [16]:
package_name_df

,sl,name,short_name
0,1,SUNM/PW-07,0.0
1,2,KISH/PW-30,0.0


In [17]:
boq_df_corrected

,ItemCode,Description,Unit,Rate,S1,S2U,S2D,S3,S4U,S4D,S5,S6,S7,S8U,S8D,KP30R1A,KP30R2A,KP30R3A,KP30R4B5
0,16-100,Erection of bamboo profile,No,367.41,25.000,8.000000,2.000000,39.000,1.000000,1.000000,29.000,39.000,129.000,1.000000,1.000000,2.000000,17.000000,38.000000,24.000
1,16-220-10,Earth work by carried earth (by truck/boat or ...,Cum,429.88,2668.664,3993.386000,946.300000,4294.190,145.180000,145.180000,3538.975,4238.900,16029.475,145.180000,145.180000,73.389000,1406.580000,3457.440000,2194.850
2,16-240-10,Earth work by Mechanical Excavator (Long Boon)...,Cum,157.70,2668.664,3993.386000,946.300000,4294.190,145.180000,145.180000,3538.975,4238.900,16029.475,145.180000,145.180000,73.389000,1406.580000,3457.440000,2194.850
3,16-180,Royalty of specified earth taken from private ...,Cum,16.97,2668.664,3993.386000,946.300000,4294.190,145.180000,145.180000,3538.975,4238.900,16029.475,145.180000,145.180000,73.389000,1406.580000,3457.440000,2194.850
4,40-550-30,Supplying and laying sand as filter layers FM...,Cum,1267.96,0.000,485.668312,115.087278,0.000,29.464911,29.464911,0.000,0.000,0.000,29.464911,29.464911,39.654823,499.772232,1147.155020,0.000
5,40-500-40,Supplying and placing non-woven needle punched...,Sqm,250.13,9499.066,5348.831235,1267.495553,15285.099,341.298221,341.298221,11636.046,15088.295,52704.446,341.298221,341.298221,515.503464,6187.274643,14083.900392,8697.189
6,40-530-20,Supplying and laying shingles and peagravels a...,Cum,5771.61,0.000,250.059412,59.255785,0.000,15.417314,15.417314,0.000,0.000,0.000,15.417314,15.417314,21.573801,267.350005,611.929580,0.000
7,40-530-30,Supplying and laying shingles and peagravels a...,Cum,6135.23,0.000,250.059412,59.255785,0.000,15.417314,15.417314,0.000,0.000,0.000,15.417314,15.417314,21.573801,267.350005,611.929580,0.000
8,40-190-40,Manufacturing and supplying C.C. blocks in lea...,Nos,457.33,0.000,32283.000000,7650.000000,0.000,2000.000000,2000.000000,0.000,0.000,0.000,2000.000000,2000.000000,2805.000000,35700.000000,81760.000000,0.000
9,40-190-50,Manufacturing and supplying C.C. blocks in lea...,Nos,380.95,0.000,7034.000000,1667.000000,0.000,667.000000,667.000000,0.000,0.000,0.000,667.000000,667.000000,1700.000000,17000.000000,37334.000000,0.000


In [18]:
package_name_df

,sl,name,short_name
0,1,SUNM/PW-07,0.0
1,2,KISH/PW-30,0.0


In [19]:
reach_df_corrected

,Reach,Start,Finish,Length,fill_height,type,package,SheetName,rowindex
0,S1,0.000,0.724,724,1.41,B5,SUNM/PW-07,S1,2
1,S2U,0.724,0.935,211,3.31,A,SUNM/PW-07,S2U,3
2,S2D,0.955,1.105,50,3.31,A,SUNM/PW-07,S2D,4
3,S3,1.105,2.180,1165,1.41,B5,SUNM/PW-07,S3,5
4,S4U,4.500,4.520,20,2.00,A,SUNM/PW-07,S4U,6
5,S4D,4.535,4.555,20,2.00,A,SUNM/PW-07,S4D,7
6,S5,4.600,7.800,850,1.50,B5,SUNM/PW-07,S5,8
7,S6,9.200,10.350,1150,1.41,B5,SUNM/PW-07,S6,9
8,S7,25.200,36.300,3850,1.50,B5,SUNM/PW-07,S7,10
9,S8U,35.030,35.050,20,2.00,A,SUNM/PW-07,S8U,11


In [20]:
#import reportlab.graphics.chart
help(TableStyle)

Help on class TableStyle in module reportlab.platypus.tables:

class TableStyle(builtins.object)
 |  TableStyle(cmds=None, parent=None, **kw)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, cmds=None, parent=None, **kw)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  add(self, *cmd)
 |  
 |  getCommands(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [21]:
#mypdf.build(pdf_elems)
